In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import json
from collections import Counter
import seaborn as sns

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
PATH_TO_CHAIN = '/content/drive/MyDrive/MADE/project/chains.pkl'
PATH_TO_ORDERS = '/content/drive/MyDrive/MADE/project/orders'
PATH_TO_RATING = '/content/drive/MyDrive/MADE/project/rest_ratings.pkl'

In [8]:
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype as is_datetime


def reduce_mem_usage(props: pd.DataFrame) -> pd.DataFrame:
    """
    Convert data types of a pandas DataFrame
    """
    
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if not is_datetime(props[col]):  # Exclude datetime
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            if props[col].dtype == object: 
                try:
                    props[col] = props[col].astype(np.float32)
                except Exception as e:
                    print(e)
                    continue           
            
            # make variables for Int, max and min
            IsInt = True
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                IsInt = False
#                 props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
#             asint = props[col].fillna(0).astype(np.int64)
#             result = (props[col] - asint)
#             result = result.sum()
#             if result > -0.01 and result < 0.01:
#                 IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
#             else:
#                 props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    
    return props, NAlist


In [9]:
import numpy as np
import pandas as pd
from collections import Counter
# from utils.process_data import reduce_mem_usage

# PATH_TO_CHAIN = 'C:\MADE\made-rec-sys-2021\ml-entry-exam-notebooks\data\chains.pkl'
# PATH_TO_ORDERS = 'C:\MADE\made-rec-sys-2021\ml-entry-exam-notebooks\data\orders'


def chains_eda() -> pd.DataFrame:
  path_to_files = PATH_TO_ORDERS
  tables = []
  for num in range(1, 3):
      filename = f'{path_to_files}/orders{num}.pkl'
      print(filename)
      tables.append(pd.read_pickle(filename))

  orders_df = pd.concat(tables)
  orderes_df, _ = reduce_mem_usage(orders_df)
  orders_group_df = orders_df\
  .groupby('chain_id')\
  .agg(
      {'discount_value': np.mean,
      'total_value': np.mean,
      'initial_product_sum': np.mean,
      'index': len
      }
  )
  orders_group_df.rename({'index': 'num_orders'},axis=1, inplace = True)
  orders_group_df.loc[:, 'discount_value'] = orders_group_df.discount_value /  orders_group_df.initial_product_sum
  chains_df = pd.read_pickle(PATH_TO_CHAIN)
  chains_df, _ = reduce_mem_usage(chains_df)
  chains_df.drop(
    chains_df[chains_df.chain_id.isna()].index,
    axis=0,
    inplace=True,
  )
  chains_df.drop(
    chains_df[chains_df.is_test_chain == 1].index,
    axis=0,
    inplace=True,
  )


  def count_val(col: pd.Series):
    return pd.Series([i for lst in col for i in lst])

  top_product_dict = Counter(count_val(chains_df.product_group_ids.dropna()).value_counts().to_dict(dict))
  top_cousine_dict = Counter(count_val(chains_df.product_group_ids.dropna()).value_counts().to_dict(dict))
  chains_df.product_group_ids = chains_df.product_group_ids.apply(lambda d: d if isinstance(d, list) else [])
  chains_df.loc[:,'top3_products'] = chains_df.product_group_ids.map(lambda x: any([True for i in x if i in [i[0] for i in top_product_dict.most_common(3)]]))
  chains_df.loc[:,'top1_products'] = chains_df.product_group_ids.map(lambda x: any([True for i in x if i in [i[0] for i in top_product_dict.most_common(1)]]))
  chains_df.loc[:,'top10_products'] = chains_df.product_group_ids.map(lambda x: any([True for i in x if i in [i[0] for i in top_product_dict.most_common(10)]]))
  chains_df.loc[:,'defaults_in_top3'] = chains_df.default_product_group_id.map(lambda x: True if x in [i[0] for i in top_product_dict.most_common(3)] else False)
  chains_df.loc[:,'vendors_in_chain'] = chains_df.groupby('chain_id')['vendor_id'].transform(len)
  chains_df.drop([
       'vendor_id', 'chain_name', 'chain_name_en',
       'chain_created_at', 'vendor_created_at',
       'category_id', 'default_product_group_id', 'product_group_ids',
       'cuisine_ids', 'food_for_points_marker', 'takeaway_support',
       'is_test_chain', 'citymobil_support',
       'deliveryprovider_id', 'lat', 'lon', 'chain_active',
        'chain_deleted', 'vendor_deleted', 'city_id', 'is_qsr',
       ],axis=1, inplace=True)
  chains_group_df = chains_df.groupby('chain_id').agg(np.mean)
  df = chains_group_df.join(orders_group_df, how='inner')
  df.loc[:, 'order_per_vendor'] = df.num_orders / df.vendors_in_chain
  return df

In [10]:
df = chains_eda()

/content/drive/MyDrive/MADE/project/orders/orders1.pkl
/content/drive/MyDrive/MADE/project/orders/orders2.pkl
chain
Memory usage of properties dataframe is : 25.69790744781494  MB
******************************
Column:  chain_id
dtype before:  float64
dtype after:  float64
******************************
******************************
Column:  vendor_id
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  chain_name
dtype before:  object
could not convert string to float: 'Такаши'
******************************
Column:  chain_name_en
dtype before:  object
could not convert string to float: 'takashi_sochi_vd'
******************************
Column:  chain_active
dtype before:  float64
dtype after:  float64
******************************
******************************
Column:  chain_deleted
dtype before:  float64
dtype after:  float64
******************************
******************************
Column:  vendor_deleted
dtype befor